In [24]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
res= requests.get('https://boards.greenhouse.io/embed/job_board?for=coursera')
print(res.text)

<!DOCTYPE html>
<html lang="en">
<head prefix="og: http://ogp.me/ns#">
<script type="text/javascript">var _rollbarConfig = {"accessToken":"333f38f9343644929613c6ff7e965f21","captureUncaught":true,"payload":{"environment":"production"},"hostBlockList":["linkedin.com"],"checkIgnore":function(isUncaught, args, payload) {
  try {
    var isPhantomJS = window.navigator.userAgent.match(/phantomjs/i);
    var isSplash= window.navigator.userAgent.match(/splash version/i);
    var firefox = window.navigator.userAgent.match(/(firefox)\/(\w+)/i); // JSON, must escape slash

    var isFirefoxLessThan50;
    if (firefox) {
      var majorVersion = Number(firefox[2]);
      isFirefoxLessThan50 = majorVersion < 50
    }

    return Boolean(isPhantomJS) || Boolean(isSplash) || Boolean(isFirefoxLessThan50);
  } catch (e) {
    return true;
  }
}
};</script>
<script type="text/javascript">!function(r){var e={};function o(n){if(e[n])return e[n].exports;var t=e[n]={i:n,l:!1,exports:{}};return r[n].call(t.

In [3]:
soup = BeautifulSoup(res.text, "html.parser")

In [28]:
job_urls_class = soup.findAll('div',class_='opening',recursive=True)
# print(job_urls_class)
df=pd.DataFrame(columns=['Job Position','job_URL',"location","Full Decription",'job overview','responsibilities','basic qualifications','preferred qualifications'])
for class_ in job_urls_class:
    position=class_.find('a').text            
    print("position : ",class_.find('a').text)
    print("Job Url : ",class_.find('a').get('href'))
    url=class_.find('a').get('href')
    token = url.split("=")[1]
    url="https://boards.greenhouse.io/embed/job_app?for=coursera&token="+str(token)
    
    job_page=requests.get("https://boards.greenhouse.io/embed/job_app?for=coursera&token="+str(token))
#     print(job_page.text)
    job_page=BeautifulSoup(job_page.text, "html.parser")
    location=job_page.find('div',class_='location').text.strip()
    print("location : ",job_page.find('div',class_='location').text.strip())
    job_content = job_page.find('div',attrs={'id':'content'}).text
#     print(job_content)
    children = job_page.find('div',attrs={'id':'content'}).findChildren( recursive=False)
    tags=['job overview','responsibilities','basic qualifications','preferred qualifications']
    data={}
    data['Job Overview']=""
    data['Responsibilities']=""
    data['Basic Qualifications']=""
    data['Preferred Qualifications']=""
    for ind,child in enumerate(children):        
#         print(child.text)
        if child.text.strip().replace(":","").lower() in tags:
#             print(child.text,children[ind+1].text)
            data[child.text.strip().replace(":","")]=children[ind+1].text
                 
#     print(data) 
    
    df.loc[len(df.index)] = [position,url,location,job_content,data['Job Overview'],data['Responsibilities'],data['Basic Qualifications'],data['Preferred Qualifications']]             
    
    print("-"*50)
#     break
print(df)

position :  Content Strategy Manager, Computer Science & Technology
Job Url :  https://about.coursera.org/careers/jobs?gh_jid=4645401004
location :  United States
--------------------------------------------------
position :  Engineering Manager
Job Url :  https://about.coursera.org/careers/jobs?gh_jid=4614435004
location :  Bulgaria
--------------------------------------------------
position :  Senior Software Engineer, Back End
Job Url :  https://about.coursera.org/careers/jobs?gh_jid=4710854004
location :  Bulgaria
--------------------------------------------------
position :  Sr Software Engineer - Application Architecture
Job Url :  https://about.coursera.org/careers/jobs?gh_jid=4606915004
location :  Canada
--------------------------------------------------
position :  Equity Accountant
Job Url :  https://about.coursera.org/careers/jobs?gh_jid=4703409004
location :  United States
--------------------------------------------------
position :  Senior Data Scientist, Analytics
Job U

In [31]:
df.to_excel(r'C:\Users\prati\Desktop\cooking videos\Result.xlsx',index=False)#save the result